# System rekomendacyjny dla filmów

Celem tego projektu jest stworzenie systemu rekomendacyjnego dla wyszukiwarki filmów. Przedstawmy sytuację: klient zwraca się do nas z prośbą o polecenie filmu do obejrzenia, ale nie jest pewien, jaki konkretnie wybrać.

W naszym systemie rekomendacyjnym opieramy się na idei filtrowania kolaboratywnego (ang. collaborative filtering).

## Idea filtrowania kolaboratywnego

Podczas wyboru filmu, który może spodobać się naszemu użytkownikowi, musimy zastanowić się, jakie dane mamy do dyspozycji. Przede wszystkim posiadamy informacje na temat ocen, które nasz użytkownik wystawiał filmom, które już obejrzał. Warto zaznaczyć, że nie posiadamy informacji o wszystkich filmach w naszej bazie danych, lecz najczęściej mamy dostęp tylko do niewielkiego podzbioru kilku lub kilkunastu filmów. Dzięki temu możemy dowiedzieć się, które filmy podobały się naszemu użytkownikowi, a które nie.

Dodatkowo, posiadamy również informacje o preferencjach innych użytkowników. Możemy więc znaleźć w danych grupę użytkowników, którzy mają podobny gust filmowy do naszego użytkownika. Warto zauważyć, że praktycznie każdy z tych innych użytkowników obejrzał filmy, których nasz użytkownik jeszcze nie widział. Idea filtrowania kolaboratywnego jest prosta: jeśli inny użytkownik o podobnych gustach wysoko ocenił jakiś film, to nasz użytkownik prawdopodobnie też go wysoko oceni. Zatem rekomendujemy filmy, które użytkownicy o podobnych gustach wysoko ocenili.

### Wstępna eksploracja danych
Wczytujemy zawartość pliku, który zawiera dane oraz wyświetlamy zawartość wczytanej ramki.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('filmy.csv', sep=';', decimal=',')

print(df)

                                                filmy  1648  5136  918  2824   
0       11: Star Wars: Episode IV - A New Hope (1977)   NaN   4.5  5.0   4.5  \
1                             12: Finding Nemo (2003)   NaN   5.0  5.0   NaN   
2                             13: Forrest Gump (1994)   NaN   5.0  4.5   5.0   
3                          14: American Beauty (1999)   NaN   4.0  NaN   NaN   
4   22: Pirates of the Caribbean: The Curse of the...   4.0   5.0  3.0   4.5   
..                                                ...   ...   ...  ...   ...   
94                              9802: The Rock (1996)   5.0   NaN  NaN   NaN   
95                       9806: The Incredibles (2004)   3.5   5.0  3.5   NaN   
96                 10020: Beauty and the Beast (1991)   3.0   5.0  NaN   NaN   
97                                36657: X-Men (2000)   NaN   4.5  NaN   NaN   
98                     36658: X2: X-Men United (2003)   3.5   4.0  NaN   NaN   

    3867  860  3712  2968  3525  ...  3

Wypiszmy nazwy kolumn ramki danych (oprócz pierwszej), aby poznać ID użytkowników w bazie danych.

In [4]:
column_names = df.columns[1:]
print(column_names)

Index(['1648', '5136', '918', '2824', '3867', '860', '3712', '2968', '3525',
       '4323', '3617', '4360', '2756', '89', '442', '3556', '5261', '2492',
       '5062', '2486', '4942', '2267', '4809', '3853', '2288'],
      dtype='object')


Wyświetlmy kolumnę danych z ocenami filmów konkretnego użytkownika.

In [5]:
user_id = 5136 # np. dla użytkownika o id 5136
ratings_column = df[str(user_id)]
print(ratings_column)

0     4.5
1     5.0
2     5.0
3     4.0
4     5.0
     ... 
94    NaN
95    5.0
96    5.0
97    4.5
98    4.0
Name: 5136, Length: 99, dtype: float64


### Znajdowanie użytkowników o podobnych gustach

Spróbujemy teraz wypracować rekomendacje dla użytkownika o danym id. 

Zacznijmy od policzenia korelacji ocen tego użytkownika z innym użytkownikiem funkcją `pearsonr`.

Funkcja `pearsonr` zwraca krotkę dwóch wartości. Pierwsza wartość to korelacja, druga to wynik testu istotności. W tym ćwiczeniu wynik testu statystycznego zostanie pominięty.

In [2]:
from scipy.stats import pearsonr 

user1_id = 89 # id użytkownika, dla którego tworzymy rekomendację 
user2_id = 3867

user1_ratings = np.nan_to_num(df[str(user1_id)])
user2_ratings = np.nan_to_num(df[str(user2_id)])

correlation, _ = pearsonr(user1_ratings, user2_ratings)

print("Korelacja (przy uwzględnieniu pustych pól) między użytkownikiem o id", user1_id, "i użytkownikiem o id", user2_id, "wynosi w zaokrągleniu:", round(correlation, 3))

Korelacja (przy uwzględnieniu pustych pól) między użytkownikiem o id 89 i użytkownikiem o id 3867 wynosi w zaokrągleniu: 0.267


Niestety nie udało się uzyskać oczekiwanego wyniku, ponieważ niektóre z wartości w kolumnach są puste. Wynika to z faktu, że dani użytkownicy nie oglądali wszystkich filmów w bazie danych, a jedynie niektóre z nich.
Musimy przeiterować po wartościach kolumn, zachować tylko te filmy do których mamy oceny i na tak ograniczonym zbiorze policzyć korelację.

In [3]:
def cor(col1, col2):
    valid_indices = np.logical_and(~np.isnan(col1), ~np.isnan(col2))
    valid_col1 = col1[valid_indices]
    valid_col2 = col2[valid_indices]
    correlation, _ = pearsonr(valid_col1, valid_col2)
    return correlation

user1_id = 89
user2_id = 3867

user1_ratings = df[str(user1_id)]
user2_ratings = df[str(user2_id)]

correlation = cor(user1_ratings, user2_ratings)

print("Korelacja (przy ograniczonym zbiorze) między użytkownikiem o id", user1_id, "i użytkownikiem o id", user2_id, "wynosi w zaokrągleniu:", round(correlation, 3))

Korelacja (przy ograniczonym zbiorze) między użytkownikiem o id 89 i użytkownikiem o id 3867 wynosi w zaokrągleniu: -0.004


Napiszmy teraz funkcję, która dla danego użytkownika znajduje 5 najbardziej skorelowanych (podobnych) użytkowników. Wyjściem funkcji jest lista par (korelacja, id użytkownika).

In [4]:
def get_users(df, user):
    user_ratings = df[str(user)]
    correlations = []
    
    for column in df.columns[1:]:
        if column != str(user):
            other_user_ratings = df[column]
            correlation = cor(user_ratings, other_user_ratings)
            correlations.append((correlation, int(column)))
    
    correlations.sort(reverse=True)
    top_users = correlations[:5]
    
    return top_users

user_id = 860 # id uzytkownika, dla którego szukamy 5 najbardziej skolerowanych.
top_users = get_users(df, user_id)

print("Najbardziej skorelowani użytkownicy dla użytkownika o id", user_id)
for correlation, user in top_users:
    print("Id użytkownika:", user, "--> Korelacja w zaokrągleniu:", round(correlation, 3))

Najbardziej skorelowani użytkownicy dla użytkownika o id 860
Id użytkownika: 89 --> Korelacja w zaokrągleniu: 0.539
Id użytkownika: 1648 --> Korelacja w zaokrągleniu: 0.481
Id użytkownika: 918 --> Korelacja w zaokrągleniu: 0.468
Id użytkownika: 5261 --> Korelacja w zaokrągleniu: 0.399
Id użytkownika: 3525 --> Korelacja w zaokrągleniu: 0.388


### Predykcja oceny filmu

Zaproponuję teraz funkcję, która zwróci listę par (przewidywana ocena, film), gdzie przewidywana ocena jest obliczana jako średnia ważona ocen podobnych użytkowników, z wagami odpowiadającymi podobieństwu (korelacji) między nimi.

Wzór na predykcję oceny naszego użytkownika do $j$-tego filmu:
$$\hat{y_j} = \frac{\sum_{i \in U} w_iy_{ij}}{\sum_{i \in U} w_i}$$
gdzie 
$U$ zawiera indeksy podobnych użytkowników, którzy ocenili $j$-ty film,  
$y_{ij}$ to ocena $j$-tego filmu przez $i$-tego użytkownika,  
$w_i$ to podobieństwo (korelacja) $i$-tego użytkownika do analizowanego użytkownika.

In [18]:
def get_marks(df, user):
    user_ratings = df[str(user)]
    similar_users = get_users(df, user)
    marks = []
    
    for index, row in df.iterrows():
        film_id = row['filmy']
        if np.isnan(user_ratings[index]):
            weighted_sum = 0
            similarity_sum = 0
            
            for correlation, similar_user in similar_users:
                similar_user_ratings = df[str(similar_user)]
                
                if not np.isnan(similar_user_ratings[index]):
                    weighted_sum += correlation * similar_user_ratings[index]
                    similarity_sum += correlation
            
            if similarity_sum > 0:
                estimated_rating = weighted_sum / similarity_sum
                marks.append((estimated_rating, film_id))
    
    marks.sort(reverse=True)
    return marks

Wywołajmy teraz napisaną funkcję dla konkretnego użytkonika, aby zobaczyć, jakie filmy powinniśmy zaproponować temu użytkownikowi.

In [20]:
user_id = 89
estimated_marks = get_marks(df, user_id)

print("Estymacja oceny filmów dla użytkownika o id", user_id)
for i, (estimated_rating, film) in enumerate(estimated_marks):
    print("Film:", film, "--> Estymacja oceny w zaokrągleniu:", round(estimated_rating, 5))
    if i == 2:  # Wyświetl tylko pierwsze 3 rekordy
        break

Estymacja oceny filmów dla użytkownika o id 89
Film: 807: Seven (a.k.a. Se7en) (1995) --> Estymacja oceny w zaokrągleniu: 4.77409
Film: 424: Schindler's List (1993) --> Estymacja oceny w zaokrągleniu: 4.72906
Film: 122: The Lord of the Rings: The Return of the King (2003) --> Estymacja oceny w zaokrągleniu: 4.69596


## Podsumowanie projektu i wnioski

W ramach tego projektu udało nam się stworzyć własny system rekomendacyjny filmów, który opiera się na ocenach innych użytkowników oraz korelacji między nimi. Dzięki temu systemowi możemy otrzymywać rekomendacje filmowe, które najprawdopodobniej będą nam odpowiadać.

Jednakże, implementacja tego systemu w rzeczywistości wiąże się z pewnymi trudnościami i wymaganiami. Jednym z głównych wyzwań jest zebranie odpowiedniego zbioru danych. Aby system działał efektywnie, potrzebujemy jak największej ilości danych, co może być trudne do osiągnięcia.

Ważne aspekty do uwzględnienia podczas implementacji takiego systemu to prywatność i bezpieczeństwo danych. Musimy zapewnić odpowiednie zabezpieczenia danych użytkowników i zagwarantować im, że ich prywatność nie będzie naruszona.

Ponadto, warto zauważyć, że brak początkowej bazy danych stanowi kolejne wyzwanie. Dopiero w miarę rozwoju systemu i zebrania odpowiedniej ilości danych od użytkowników będziemy mieli satysfakcjonujące wyniki rekomendacji.

Ważne jest również zrozumienie, że choć obliczyliśmy, jak bardzo użytkownik może polubić dany film na podstawie podobieństwa gustów, nie oznacza to automatycznie, że oceni go tak samo w rzeczywistości. Indywidualne preferencje użytkowników mogą wpływać na ostateczną ocenę filmu.